Running record of prior work:
*   https://arxiv.org/pdf/2307.10236


Self-reported uncertainty:
*   https://aclanthology.org/2024.acl-long.283.pdf - primary inspiration for our method
*   https://arxiv.org/pdf/2207.05221 - Earlier method, iterated on by 2024.acl-long.283
*   https://aclanthology.org/2024.emnlp-main.299.pdf - only applies to long form (multi-sentence) generation. Also requires multiple long-form responses to compute the uncertainty for a single response.
*   https://aclanthology.org/2024.emnlp-main.1205.pdf - sentence level, requires multisampling, SAR scores are on shaky theoretical grounding

# Installs and Imports

In [32]:
import csv
import itertools
import json
import tqdm
import os
import ast
import gc
import numpy as np
import pandas as pd
import scipy
import time
import accelerate
import pickle
import io

import torch
import transformers as tf

In [ ]:
my_token = 'INSERT JUPYTER NOTEBOOK TOKEN HERE'

# Loading Functions

In [ ]:
def load_data(filenames):
    ctx = filenames['Question']
    cmplts = filenames['Answers']

    contexts = []
    for i in ctx:
      contexts.append(i)

    completions = []
    for i in cmplts:
      completions.append(i)

    return contexts, completions


def load_model(model_name, device='cuda', token=None, verbose=False):
    config = tf.AutoConfig.from_pretrained(model_name, token=my_token)
    tokenizer = tf.AutoTokenizer.from_pretrained(model_name, token=my_token)

    tokenizer.pad_token = tokenizer.eos_token
    
    try:
        transformer = tf.AutoModelForCausalLM.from_pretrained(model_name,
                                                   config=config,
                                                   token=my_token,
                                                   resume_download=True,
                                                   low_cpu_mem_usage=True,
                                                   device_map="auto")
        if verbose:
            print(f'Successfully loaded model: {model_name}')
    except Exception as e:
        if verbose:
            print(f'[ERROR] Failed to load model: {model_name}')
        raise e

    transformer.pop_ready = False

    return transformer, tokenizer

# Uncertainty Functions

In [ ]:
def next_token_pdf(model, tokenizer, ctx):
    tokenized = tokenizer(ctx, return_tensors='pt', padding=True, add_special_tokens=False).to(model.device.type)
    outputs = model(**tokenized)

    return torch.nn.LogSoftmax(dim=-1)(outputs.logits[0, -1, :]).exp().detach()

def cloze_test(model, tokenizer, pdf, tok_cmplts):
    probs = pdf[tok_cmplts['input_ids'][:,0]]

    return np.argmax(probs.detach().cpu().numpy())

def unc_self_report(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt):
    '''
      result is a list of shape (cmplts_count, 2)

      [
        answer choice 1 results = [prob(best), prob(worst)],
        answer choice 2 results = [prob(best), prob(worst)],
        ...
      ]
    '''
    choice_uqs = []
    unc_cmplts = ['best', 'worst']
    uq_cmplts_tok = tokenizer(unc_cmplts, return_tensors='pt', padding=True, add_special_tokens=False)
    for c in cmplts:
        uncertainty_prompt = f'{c}\nThis answer is '
        uq_pdf = next_token_pdf(model, tokenizer, ctx + uncertainty_prompt)
        choice_uqs.append(list(uq_pdf[uq_cmplts_tok['input_ids'][:,0]].detach().cpu().numpy()))
    return choice_uqs

def unc_frequency(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt, trials=100):
    tok_cmplts = tokenizer(cmplts, return_tensors='pt', padding=True, add_special_tokens=False)
    choice_uqs = list(output_pdf[tok_cmplts['input_ids'][:,0]].detach().cpu().numpy() * trials)

    return choice_uqs

def unc_cred_int(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt, p=0.95):
    sorted_probs, indices = torch.sort(output_pdf, dim=-1, descending=True)
    cum_sum_probs = torch.cumsum(sorted_probs, dim=-1)
    nucleus = torch.count_nonzero(cum_sum_probs < p)

    return nucleus.item()

def unc_entropy(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt, choices_only=False):
    if choices_only:
        tok_cmplts = tokenizer(cmplts, return_tensors='pt', padding=True, add_special_tokens=False)
        choices_pdf = output_pdf[tok_cmplts['input_ids'][:,0]]
        return np.sum(choices_pdf.detach().cpu().numpy() * np.log(choices_pdf.detach().cpu().numpy()))
    else:
        return np.sum(output_pdf.detach().cpu().numpy() * np.log(output_pdf.detach().cpu().numpy()))

def unc_choice_entropy(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt):
    return unc_entropy(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt, choices_only=True)

def unc_topk_entropy(model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt, k=10):
    topk = torch.topk(output_pdf, k)[0].detach().cpu().numpy()

    return np.sum(topk * np.log(topk))

def get_uncertainty(model, tokenizer, ctx, cmplts, uncertainty_type, output_pdf, choice, choice_txt, **kwargs):
    return unc_funcs[uncertainty_type](model, tokenizer, ctx, cmplts, output_pdf, choice, choice_txt, **kwargs)

def next_token_pdf_alt(model, tokenizer, ctx):
    tokenized = tokenizer(ctx, return_tensors='pt', padding=True, add_special_tokens=False).to(model.device.type)
    outputs = model(**tokenized)

    return torch.nn.LogSoftmax(dim=-1)(outputs.logits[:, -1, :]).exp().detach()

unc_funcs = {
    'self-report' : unc_self_report,
    'freq'        : unc_frequency,
    'top-p'       : unc_cred_int,
    'entropy'     : unc_entropy,
    'c-ent'       : unc_choice_entropy,
    'top-k-ent'   : unc_topk_entropy,
}


# Main Functions

In [ ]:
def prepare_model_for_pop(model):
    pop.DropoutUtils.add_new_dropout_layers(model)
    pop.DropoutUtils.convert_dropouts(model)
    pop.DropoutUtils.activate_mc_dropout(model, activate=True, random=0.1)

    model.pop_ready = True

def run_experiment(model,
                   tokenizer,
                   data_locs,
                   output_dir,
                   causal=True,
                   device='cuda',
                   token=None,
                   early_exit=False,
                   verbose=False,
                   **kwargs):
    #copy original table
    df = pd.read_pickle(data_locs)

    if early_exit:
        df = df.head(3)

    #add model output and pdf columns to df
    df["model output"] = None
    df["pdf"] = None
    
    #create output file. if it exists, append with new number
    version = 1
    while (os.path.exists(output_dir)):
        if '(' in output_dir:
            output_dir = output_dir[:1] + str(version + 1) + output_dir[2:]
        else:
            output_dir = '(1)' + output_dir
        version += 1
    df.to_pickle(output_dir)
    
    #instructional content to be prepended to contexts
    instr_str = "Following is a question and a selection of answer choices. Provide the label for the answer with which you most agree.\nQuestion: "
    query_str = "\nAnswer: "
    index = 0
    
    #make cmplt context lists
    quest_fmt = "\n {}. {}"
    contexts, completions = load_data(df)
    indices = 0
    for cmplt, ctx in zip(completions, contexts):
        cmplt = str(cmplt).split(",")
        for index, j in enumerate(cmplt):
            letter = chr(index + 65)
            ctx = ctx + quest_fmt.format(letter, j)
            cmplt[index] = letter
        completions[indices] = cmplt
        contexts[indices] = instr_str + ctx + query_str
        indices += 1
        
    index = 0
    
    #
    for ctx, cmplt in zip(contexts, completions):
        
        tok_cmplts = tokenizer(cmplt, return_tensors='pt', padding=True, add_special_tokens=False)
        output_pdf = next_token_pdf(model, tokenizer, ctx)
        choice = cloze_test(model, tokenizer, output_pdf, tok_cmplts)
        choice_txt = cmplt[choice]

        #append answer to answer column in df
        df.at[index, 'model output'] = choice_txt
        
        #append pdf to pdf column in df
        df.at[index, 'pdf'] = output_pdf
        
        #save file every line
        df.to_pickle(output_dir)

        percent = round((index / 3000) * 100, 3)
        print(f'\tsaved {percent}% of lines to {output_dir}', end='\r')
        
        index += 1
        
    #save df as pickle
    df.to_pickle(output_dir)



In [ ]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)
 
def load_df(filename) -> pd.DataFrame:
    with open(filename, 'rb') as f:
        df = CPU_Unpickler(f).load()
   
    return df
   
def norm_ent(probs):
    return scipy.stats.entropy(probs) / np.log(len(probs))
 
def get_top_p(sorted_probs, p):
    cum_sum = np.cumsum(sorted_probs)
    k = np.searchsorted(cum_sum, p, side='right')
    return sorted_probs[:k]
 
def get_metrics(df, dataset_mode, model_name, hf_token):
    '''
    dataset_mode:
        0 = mmlu, 1 = roper
 
    Implemented metrics:
        top-k:
            k = 5, 10, 25, 50, 100
            entropy
        top-p:
            p = 0.95, 0.9, 0.75, 0.5
            entropy
            size
        total entropy
        choices:
            entropy
            raw probs
        non-entropy:
            top-1 prob
    '''
    metric_df = pd.DataFrame()
 
    # choices first (before sorting)
    tokenizer = tf.AutoTokenizer.from_pretrained(model_name, token=hf_token)
    tokenizer.pad_token = tokenizer.eos_token
    choices = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    choices = tokenizer(choices, return_tensors='np', padding=False, add_special_tokens=False)['input_ids'][:,0]
 
#     metrics_df['pdf'] = df.apply(lambda x: x['pdf'].to_device('cuda'))

    if dataset_mode == 0:
        metric_df['choice-probs'] = df.apply(lambda x: x['pdf'][choices[:4]].tolist(), axis=1)
    elif dataset_mode == 1:
        metric_df['choice-probs'] = df.apply(lambda x: x['pdf'][choices[:len(x['Answer_Ratios'])]].tolist(), axis=1)
    elif dataset_mode == 2:
        metric_df['choice-probs'] = df.apply(lambda x: x['pdf'][choices[:len(x['Answers'].split(','))]].tolist(), axis=1)
    
    metric_df['choice-ent'] = metric_df['choice-probs'].apply(lambda x: norm_ent(x))
 
    #sort tensor
    sorted_df = pd.DataFrame()
    sorted_df['pdf'] = df['pdf'].apply(lambda x: np.sort(x)[::-1])
 
    # non-entropy
    metric_df['top-1-abs'] = sorted_df['pdf'].apply(lambda x: x[0])
 
    # total-entropy
    metric_df['total-ent'] = sorted_df['pdf'].apply(lambda x: norm_ent(x))
 
 
    # top-k
    for k in [5, 10, 25, 50, 100]:
        metric_df[f'top-k-ent-{k}'] = sorted_df['pdf'].apply(lambda x: norm_ent(x[:k]))
 
    # top-p
    for p in [0.95, 0.9, 0.75, 0.5]:
        top_p = sorted_df['pdf'].apply(lambda x: get_top_p(x, p))
        metric_df[f'top-p-len-{p}'] = top_p.apply(lambda x: len(x))
        metric_df[f'top-p-ent-{p}'] = top_p.apply(lambda x:
                                                0 if len(x) == 1 else     # no uncertainty when only one element qualifies
                                                1 if len(x) == 0 else     # complete uncertainty when no elements qualify
                                                norm_ent(x)
                                                )
 
    df = pd.concat([df, metric_df], axis=1)
    df = df.drop('pdf', axis=1)
    return df
 

llama-3.1 1,3,8
solar
mistral
mistral 2

In [ ]:
models = [
     'meta-llama/Llama-3.2-1B',
     'meta-llama/Llama-3.2-3B',
     'meta-llama/Llama-3.1-8B',
     'mistralai/Mistral-7B-v0.1',
     'mistralai/Mistral-7B-v0.3',
     'meta-llama/Llama-3.2-1B-Instruct',
     'meta-llama/Llama-3.2-3B-Instruct',
     'meta-llama/Llama-3.1-8B-Instruct',
     'mistralai/Mistral-7B-Instruct-v0.1',
     'mistralai/Mistral-7B-Instruct-v0.3',
]
iRoperPickles = [
    '(1)meta-llama-Llama-3.2-1B.pkl', 
    'meta-llama-Llama-3.2-3B.pkl',
    'meta-llama-Llama-3.1-8B.pkl', 
    'mistralai-Mistral-7B-v0.1.pkl',
    'mistralai-Mistral-7B-v0.3.pkl', 
    'meta-llama-Llama-3.2-1B-Instruct.pkl', 
    'meta-llama-Llama-3.2-3B-Instruct.pkl',
    'meta-llama-Llama-3.1-8B-Instruct.pkl', 
    'mistralai-Mistral-7B-Instruct-v0.1.pkl', 
    'mistralai-Mistral-7B-Instruct-v0.3.pkl', 
]

mmlu_pickles = [
    'MMLU/meta-llama-Llama-3.2-1B.pkl',
    'MMLU/meta-llama-Llama-3.2-3B.pkl',
    'MMLU/meta-llama-Llama-3.1-8B.pkl', 
    'MMLU/mistralai-Mistral-7B-v0.1.pkl',
    'MMLU/mistralai-Mistral-7B-v0.3.pkl',
    'MMLU/meta-llama-Llama-3.2-1B-Instruct.pkl', 
    'MMLU/meta-llama-Llama-3.2-3B-Instruct.pkl',
    'MMLU/meta-llama-Llama-3.1-8B-Instruct.pkl', 
    'MMLU/mistralai-Mistral-7B-Instruct-v0.1.pkl', 
    'MMLU/mistralai-Mistral-7B-Instruct-v0.3.pkl',
]

humanPickles = [
    'Human Data/meta-llama-Llama-3.2-1B (1).pkl',
    'Human Data/meta-llama-Llama-3.2-3B (1).pkl',
    'Human Data/meta-llama-Llama-3.1-8B (1).pkl',
    'Human Data/mistralai-Mistral-7B-v0.1 (1).pkl',
    'Human Data/mistralai-Mistral-7B-v0.3 (1).pkl',
    'Human Data/meta-llama-Llama-3.2-1B-Instruct (1).pkl',
    'Human Data/meta-llama-Llama-3.2-3B-Instruct (1).pkl',
    'Human Data/meta-llama-Llama-3.1-8B-Instruct (1).pkl',
    'Human Data/mistralai-Mistral-7B-Instruct-v0.1 (1).pkl',
    'Human Data/mistralai-Mistral-7B-Instruct-v0.3 (1).pkl',
]

In [33]:
for m_name, pkl_file in tqdm.tqdm(zip(models, humanPickles)):
    # 0 = mmlu, 1 = roper
    d_mode = 2

    init_df = load_df(pkl_file)

    if d_mode == 1:
        # init_df['model output'] = pd.concat([init_df['model output'][2:], init_df['model output'][:2]], axis=-1)
        # init_df['pdf']          = pd.concat([init_df['pdf'][2:], init_df['pdf'][:2]], axis=-1)
        init_df['model output'] = init_df['model output'].shift(-1)
        init_df['pdf'] = init_df['pdf'].shift(-1)

        init_df = init_df[:-2]

    elif d_mode == 2:
        init_df['model output'] = init_df['model output'].shift(-1)
        init_df['pdf'] = init_df['pdf'].shift(-1)

        init_df = init_df[:-1]

    dfMetrics = get_metrics(init_df, d_mode, m_name, my_token)
    dfMetrics.to_csv(f'{pkl_file}_human.csv')
 

10it [00:11,  1.18s/it]


In [ ]:
for m_name, pkl_file in zip(models, mmlu_pickles):
    # 0 = mmlu, 1 = roper
    d_mode = 0

    init_df = load_df(pkl_file)

    if d_mode == 1:
        # init_df['model output'] = pd.concat([init_df['model output'][2:], init_df['model output'][:2]], axis=-1)
        # init_df['pdf']          = pd.concat([init_df['pdf'][2:], init_df['pdf'][:2]], axis=-1)
        init_df['model output'] = init_df['model output'].shift(-2)
        init_df['pdf'] = init_df['pdf'].shift(-2)

        init_df = init_df[:-2]

    dfMetrics = get_metrics(init_df, d_mode, m_name, my_token)
    dfMetrics.to_csv(f'{pkl_file}_mmlu.csv')

In [ ]:
start_time = time.time()

try:
  for model_name in models:
    model, tokenizer = load_model(model_name)
    print('Running experiment on {}                                            '.format(model_name))

    data_loc_in = 'iroper_data.pkl'
    data_loc_out = model_name.replace('/','-') + '.pkl'

    run_experiment(model, tokenizer, data_loc_in, data_loc_out, verbose=True)

    print('Experiments completed: {}'.format(model_name))

    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
except Exception as e:
  stop_time = time.time()
  print('Experiment time: {}'.format(stop_time - start_time))
  raise e
else:
  stop_time = time.time()
  print('Experiment time: {}'.format(stop_time - start_time))

Running experiment on meta-llama/Llama-3.2-1B                                            


KeyboardInterrupt: 